# Hilbert Curve
## Generate points in native data space in the form of a hypercube lattice

In [1]:
"""
This is a module to convert between one dimensional distance along a
`Hilbert curve`_, :math:`h`, and N-dimensional coordinates,
:math:`(x_0, x_1, ... x_N)`.  The two important parameters are :math:`N`
(the number of dimensions, must be > 0) and :math:`p` (the number of
iterations used in constructing the Hilbert curve, must be > 0).
We consider an N-dimensional `hypercube`_ of side length :math:`2^p`.
This hypercube contains :math:`2^{N p}` unit hypercubes (:math:`2^p` along
each dimension).  The number of unit hypercubes determine the possible
discrete distances along the Hilbert curve (indexed from :math:`0` to
:math:`2^{N p} - 1`).
"""


def _binary_repr(num, width):
    """Return a binary string representation of `num` zero padded to `width`
    bits."""
    return format(num, 'b').zfill(width)


class HilbertCurve:

    def __init__(self, p, n):
        """Initialize a hilbert curve with,
        Args:
            p (int): iterations to use in the hilbert curve
            n (int): number of dimensions
        """
        if p <= 0:
            raise ValueError('p must be > 0')
        if n <= 0:
            raise ValueError('n must be > 0')
        self.p = p
        self.n = n

        # maximum distance along curve
        self.max_h = 2**(self.p * self.n) - 1

        # maximum coordinate value in any dimension
        self.max_x = 2**self.p - 1

    def _hilbert_integer_to_transpose(self, h):
        """Store a hilbert integer (`h`) as its transpose (`x`).
        Args:
            h (int): integer distance along hilbert curve
        Returns:
            x (list): transpose of h
                      (n components with values between 0 and 2**p-1)
        """
        h_bit_str = _binary_repr(h, self.p*self.n)
        x = [int(h_bit_str[i::self.n], 2) for i in range(self.n)]
        return x

    def _transpose_to_hilbert_integer(self, x):
        """Restore a hilbert integer (`h`) from its transpose (`x`).
        Args:
            x (list): transpose of h
                      (n components with values between 0 and 2**p-1)
        Returns:
            h (int): integer distance along hilbert curve
        """
        x_bit_str = [_binary_repr(x[i], self.p) for i in range(self.n)]
        h = int(''.join([y[i] for i in range(self.p) for y in x_bit_str]), 2)
        return h

    def coordinates_from_distance(self, h):
        """Return the coordinates for a given hilbert distance.
        Args:
            h (int): integer distance along hilbert curve
        Returns:
            x (list): transpose of h
                      (n components with values between 0 and 2**p-1)
        """
        if h > self.max_h:
            raise ValueError('h={} is greater than 2**(p*N)-1={}'.format(h, self.max_h))
        if h < 0:
            raise ValueError('h={} but must be > 0'.format(h))

        x = self._hilbert_integer_to_transpose(h)
        Z = 2 << (self.p-1)

        # Gray decode by H ^ (H/2)
        t = x[self.n-1] >> 1
        for i in range(self.n-1, 0, -1):
            x[i] ^= x[i-1]
        x[0] ^= t

        # Undo excess work
        Q = 2
        while Q != Z:
            P = Q - 1
            for i in range(self.n-1, -1, -1):
                if x[i] & Q:
                    # invert
                    x[0] ^= P
                else:
                    # exchange
                    t = (x[0] ^ x[i]) & P
                    x[0] ^= t
                    x[i] ^= t
            Q <<= 1

        # done
        return x

    def distance_from_coordinates(self, x_in):
        """Return the hilbert distance for a given set of coordinates.
        Args:
            x_in (list): transpose of h
                         (n components with values between 0 and 2**p-1)
        Returns:
            h (int): integer distance along hilbert curve
        """
        x = list(x_in)
        if len(x) != self.n:
            raise ValueError('x={} must have N={} dimensions'.format(x, self.n))

        if any(elx > self.max_x for elx in x):
            raise ValueError(
                'invalid coordinate input x={}.  one or more dimensions have a '
                'value greater than 2**p-1={}'.format(x, self.max_x))

        if any(elx < 0 for elx in x):
            raise ValueError(
                'invalid coordinate input x={}.  one or more dimensions have a '
                'value less than 0'.format(x))

        M = 1 << (self.p - 1)

        # Inverse undo excess work
        Q = M
        while Q > 1:
            P = Q - 1
            for i in range(self.n):
                if x[i] & Q:
                    x[0] ^= P
                else:
                    t = (x[0] ^ x[i]) & P
                    x[0] ^= t
                    x[i] ^= t
            Q >>= 1

        # Gray encode
        for i in range(1, self.n):
            x[i] ^= x[i-1]
        t = 0
        Q = M
        while Q > 1:
            if x[self.n-1] & Q:
                t ^= Q - 1
            Q >>= 1
        for i in range(self.n):
            x[i] ^= t

        h = self._transpose_to_hilbert_integer(x)
        return h

In [16]:
p = 1
N = 13
hilbert_curve = HilbertCurve(p, N)
num_points = 2 ** (p * N)
for i in range(num_points):
    coords = hilbert_curve.coordinates_from_distance(i)
    print(str(i) + " -> ", coords)


0 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
2 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
3 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
4 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
5 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
6 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
7 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
8 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
9 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]
10 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
11 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
12 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
13 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]
14 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
15 ->  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
16 ->  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
17 ->  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
18 ->  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]
19 ->  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]
20 ->  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
21 ->  [0, 0, 0, 0, 0, 

1558 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1]
1559 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0]
1560 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
1561 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]
1562 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1]
1563 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0]
1564 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
1565 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1]
1566 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
1567 ->  [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
1568 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
1569 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1]
1570 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1]
1571 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]
1572 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0]
1573 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1]
1574 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1]
1575 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]
1576 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0]
1577 ->  [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1]
1578 ->  [0, 0, 1, 0

2807 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0]
2808 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0]
2809 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1]
2810 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
2811 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0]
2812 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0]
2813 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1]
2814 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1]
2815 ->  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
2816 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2817 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]
2818 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1]
2819 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
2820 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]
2821 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1]
2822 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]
2823 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0]
2824 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0]
2825 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1]
2826 ->  [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1]
2827 ->  [0, 1, 1, 1

4307 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0]
4308 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0]
4309 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1]
4310 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1]
4311 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0]
4312 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0]
4313 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1]
4314 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1]
4315 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0]
4316 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0]
4317 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1]
4318 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1]
4319 ->  [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0]
4320 ->  [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
4321 ->  [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]
4322 ->  [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1]
4323 ->  [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
4324 ->  [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]
4325 ->  [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1]
4326 ->  [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
4327 ->  [1, 1, 0, 0

5807 ->  [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
5808 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0]
5809 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1]
5810 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]
5811 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0]
5812 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0]
5813 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1]
5814 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1]
5815 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
5816 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0]
5817 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1]
5818 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]
5819 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0]
5820 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0]
5821 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1]
5822 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]
5823 ->  [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
5824 ->  [1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]
5825 ->  [1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]
5826 ->  [1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1]
5827 ->  [1, 1, 1, 0

7307 ->  [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]
7308 ->  [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0]
7309 ->  [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1]
7310 ->  [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]
7311 ->  [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]
7312 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0]
7313 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1]
7314 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
7315 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0]
7316 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0]
7317 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1]
7318 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1]
7319 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0]
7320 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0]
7321 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
7322 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1]
7323 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0]
7324 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
7325 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
7326 ->  [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1]
7327 ->  [1, 0, 0, 1

In [21]:
import numpy as np
import pandas as pd
import time
import sklearn


readdata = pd.read_csv("/home/dom/Documents/MPhys/TheGrandTour/wine_data.txt", sep="\t", header=None);
data = np.array(readdata);
data = np.delete(data, 0, 0)
data = data.astype(float)
data = np.swapaxes(data,0,1)

classification = data[13]
data = np.delete(data, 13, axis=0)


In [22]:
len(data)

13

In [24]:
for i in range(0, np.shape(data)[0]):
    data[i,:] = (data[i,:] / np.ndarray.max(data[i,:])) * 2 - 1

In [34]:
min(data[2])

-0.1578947368421052

In [27]:
midpoint = []
for i in range(len(data)):
    ave = np.mean(data[i])
    midpoint.append(ave)
    

In [28]:
midpoint

[0.7532863085000797,
 -0.19436265013560633,
 0.46533551327095,
 0.2996629213483146,
 0.23137744486059092,
 0.18304760801575354,
 -0.20107493585773686,
 0.09652706843718074,
 -0.11122967798631601,
 -0.22183232497839245,
 0.1198239043301137,
 0.3058426966292135,
 -0.1108413590155163]